In [8]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
import re
import json
import os
from tqdm import tqdm

def compare_ignore_spaces(str1, str2):
    """
    Compares two strings while ignoring spaces between characters.
    Returns True if they are equal when spaces are removed, otherwise False.
    """
    return str1.replace(" ", "") == str2.replace(" ", "")

def extract_final_answer(text, marker="Answer:"):
    """
    Extract the final answer following the specified marker.
    """
    if marker in text:
        return text.split(marker)[-1].strip()
    return None
import re

def extract_boxed_content(text):
    """
    Extracts all content enclosed within curly brackets {} only if it appears in the format \(\boxed{...}\).
    
    :param text: The input string containing content in the boxed format.
    :return: A list of extracted content that is inside the curly brackets within \(\boxed{...}\).
    """
    # Check for boxed content first across the entire text
    if "oxed" in text:
        boxed_match = re.search(r'oxed\s*\{([^}]*)\}', text)
        if boxed_match:
            try:
                boxed_content = boxed_match.group(1)
                try:
                    return float(boxed_content)
                except ValueError:
                    return boxed_content
            except AttributeError:
                pass
    
    # If no boxed content found, look for numbers in the last meaningful sentence
    # Split by sentence endings, but keep non-empty segments
    sentence_pattern = r'(?<!\d)[.!?](?!\d)'
    sentences = [s.strip() for s in re.split(sentence_pattern, text) if s.strip()]
    
    # Get the last non-empty sentence
    if sentences:
        last_sentence = sentences[-1]
        numbers = re.findall(r'[-+]?\d*\.?\d+', last_sentence)
        if numbers:
            return float(numbers[-1])
    
    # If we found no boxed content and no numbers in the last sentence,
    # try to find any numbers in the entire text as a fallback
    all_numbers = re.findall(r'[-+]?\d*\.?\d+', text)
    if all_numbers:
        return float(all_numbers[-1])
    
    # No valid answer found
    return None
       

def test_model_on_gsm8k(model, tokenizer, target_device, num_examples=500, is_correct = False):
    """
    Test the model on the GSM8K dataset.
    """
    # Load the GSM8K test split
    dataset = load_dataset("openai/gsm8k", "main", split="test")
    num_examples=len(dataset)

    
    # Force model to use a single device
    model = model.to(target_device)
    
    results = []
    total = 0
    correct = 0
    system_prompt = "Please reason step by step, and conclude your response with 'Answer: [final answer]'."
    
        
    for example in tqdm(dataset.select(range(num_examples)), desc="Evaluating GSM8K"):
        question = example["question"]
        ground_truth_answer = example["answer"].split("####")[-1].strip()
        try:
            ground_truth_answer=float(ground_truth_answer)
        except:
            ground_truth_answer=ground_truth_answer
        
        # Construct chat-style prompt using instruct format
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": question}
        ]
        prompt_text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        
        try:
            # Clear CUDA cache to free up memory
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
            
            # Make sure inputs are on the target device
            inputs = tokenizer(prompt_text, return_tensors="pt").to(target_device)
            
            with torch.no_grad():
                # Generate the outputs
                outputs = model.generate(
                    **inputs,
                    max_new_tokens=512,
                    temperature=0.0,
                    do_sample=False
                )
            
            # Make sure outputs are on the target device if needed
            if outputs.device != target_device:
                outputs = outputs.to(target_device)
                
            generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
            predicted_answer = extract_final_answer(generated)
            
            
            # Normalize answers for comparison
            
            num_predicted = extract_boxed_content(predicted_answer)
            
    
            #num_predicted=float(num_predicted)
            #ground_truth = float(ground_truth_answer)
                

            if isinstance(num_predicted, str) and isinstance(ground_truth_answer, str):
                if compare_ignore_spaces(num_predicted, ground_truth_answer):
                    is_correct=True
                    correct +=1
                else:
                    is_correct=False
            else:
                if num_predicted== ground_truth_answer:
                    is_correct=True
                    correct +=1
                else: 
                    is_correct=False
            
            results.append({
                "question": question,
                "ground_truth": ground_truth_answer,
                "predicted": predicted_answer,
                "normalized_predicted": num_predicted,
                "is_correct": is_correct
            })
            
            print(f"Example {total + 1}:")
            print("Question:", question)
            print("Ground Truth:", ground_truth_answer)
            print("num_predicted:", num_predicted)
            print("Predicted:", predicted_answer)
            print("Correct:", is_correct)
            print("-" * 50)
            
        except Exception as e:
            print(f"Error processing example {total + 1}: {e}")
            results.append({
                "question": question,
                "ground_truth": ground_truth_answer,
                "predicted": "ERROR",
                "is_correct": False,
                "error": str(e)
            })
        
        total += 1
    
    accuracy = correct / total if total > 0 else 0
    print(f"GSM8K Accuracy on {total} examples: {accuracy:.4f}")
    
    # Save results to file
    with open("gsm8k_results.json", "w") as f:
        json.dump({
            "accuracy": accuracy,
            "examples": results
        }, f, indent=2)
    
    return accuracy, results

def test_model_on_math(model, tokenizer, target_device, num_examples=500, is_correct=False):
    """
    Test the model on the MATH dataset.
    """
    # Force model to use a single device
    model = model.to(target_device)
    
    # Try to load the MATH dataset with error handling
    try:
        dataset = load_dataset("math-ai/math500", split="test")
    except Exception as e:
        print(f"Error loading MATH dataset: {e}")       
    
    
    num_examples = len(dataset)
    print("num_examples", num_examples)
    
    print(f"Limiting to {num_examples} examples (max available in dataset)")
    
    results = []
    total = 0
    correct = 0
    correct_by_level = {level: {"correct": 0, "total": 0} for level in range(1, 6)}
    correct_by_type = {}
    
    system_prompt = "You are a mathematics expert. Please solve the following problem step by step, showing your work clearly. Conclude with 'Answer: [your final answer]'."
    
    for i, example in enumerate(tqdm(dataset.select(range(num_examples)), desc="Evaluating MATH")):
        # Handle different dataset structures
        if "problem" in example:
            question = example["problem"]
            if "solution" in example:
                ground_truth_answer = example["solution"].split("####")[-1].strip()
            
            else:
                ground_truth_answer = "Unknown"
            try:
                ground_truth_answer=float(ground_truth_answer)
                num_ground_truth=ground_truth_answer
            except:
                ground_truth_answer= ground_truth_answer
                num_ground_truth= extract_boxed_content(ground_truth_answer)
                
            level = example.get("level", 1)
            problem_type = example.get("type", "Unknown")
        else:
            # Fallback for other datasets
            if "text" in example:
                question = example["text"]
                ground_truth_answer = "N/A"
            else:
                question = f"Example {i+1}"
                ground_truth_answer = "N/A"
            level = 1
            problem_type = "Unknown"
        
        # Initialize counter for this problem type if not seen before
        if problem_type not in correct_by_type:
            correct_by_type[problem_type] = {"correct": 0, "total": 0}
        
        # Construct chat-style prompt using instruct format
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": question}
        ]
        
        try:
            # Clear CUDA cache to free up memory
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
                
            prompt_text = tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=True
            )
            
            # Make sure inputs are on the target device
            inputs = tokenizer(prompt_text, return_tensors="pt").to(target_device)
            
            with torch.no_grad():
                outputs = model.generate(
                    **inputs,
                    max_new_tokens=1024,  #MATH problems often need longer responses
                    temperature=0.0,
                    do_sample=False
                )
                
            # Make sure outputs are on the target device if needed
            if outputs.device != target_device:
                outputs = outputs.to(target_device)
                
            generated = tokenizer.decode(outputs[0], skip_special_tokens=True)

            
            predicted_answer = extract_final_answer(generated)
            num_predicted=extract_boxed_content(predicted_answer)
             
            try:
                num_predicted=float(num_predicted)
                num_ground_truth== float(num_ground_truth)
                
                if num_predicted== num_ground_truth:
                    is_correct=True
                    correct +=1
            except:
                if compare_ignore_spaces(num_predicted, num_ground_truth):
                    is_correct=True
                    correct +=1
                
                
            
            results.append({
                "question": question,
                "ground_truth": ground_truth_answer,
                "predicted": predicted_answer,
                "is_correct": is_correct,
                "level": level,
                "type": problem_type
            })
            
            print(f"Example {total + 1} (Level {level}, Type: {problem_type}):")
            print("Question:", question)
            print("Ground Truth:", ground_truth_answer)
            print("Predicted:", predicted_answer)
            print("Correct:", is_correct)
            print("-" * 50)
            
        except Exception as e:
            print(f"Error processing example {total + 1}: {e}")
            results.append({
                "question": question,
                "ground_truth": ground_truth_answer,
                "predicted": "ERROR",
                "is_correct": False,
                "level": level,
                "type": problem_type,
                "error": str(e)
            })
            correct_by_level[level]["total"] += 1
            correct_by_type[problem_type]["total"] += 1
        
        total += 1
    
    # Calculate overall accuracy
    accuracy = correct / total if total > 0 else 0
    print(f"MATH Overall Accuracy on {total} examples: {accuracy:.4f}")
    
    # Calculate and display accuracy by level
    print("\nAccuracy by difficulty level:")
    for level in range(1, 6):
        level_correct = correct_by_level[level]["correct"]
        level_total = correct_by_level[level]["total"]
        level_acc = level_correct / level_total if level_total > 0 else 0
        print(f"Level {level}: {level_acc:.4f} ({level_correct}/{level_total})")
    
    # Calculate and display accuracy by problem type
    print("\nAccuracy by problem type:")
    for p_type, counts in correct_by_type.items():
        type_correct = counts["correct"]
        type_total = counts["total"]
        type_acc = type_correct / type_total if type_total > 0 else 0
        print(f"{p_type}: {type_acc:.4f} ({type_correct}/{type_total})")
    
    # Save results to file
    with open("math_results.json", "w") as f:
        json.dump({
            "accuracy": accuracy,
            "accuracy_by_level": {level: 
                {"accuracy": correct_by_level[level]["correct"] / correct_by_level[level]["total"] 
                 if correct_by_level[level]["total"] > 0 else 0,
                 "correct": correct_by_level[level]["correct"],
                 "total": correct_by_level[level]["total"]}
                for level in range(1, 6)},
            "accuracy_by_type": {p_type: 
                {"accuracy": counts["correct"] / counts["total"] if counts["total"] > 0 else 0,
                 "correct": counts["correct"],
                 "total": counts["total"]}
                for p_type, counts in correct_by_type.items()},
            "examples": results
        }, f, indent=2)
    
    return accuracy, results




print("Starting model evaluation on GSM8K and MATH datasets...")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device count: {torch.cuda.device_count()}")
    print(f"Current CUDA device: {torch.cuda.current_device()}")
    print(f"CUDA device name: {torch.cuda.get_device_name(0)}")

# Create results directory
os.makedirs("results", exist_ok=True)

# Path to your finetuned model directory
finetuned_model_dir = "Qwen/Qwen2.5-Math-1.5B-Instruct"

# Set single device - force everything to one GPU
target_device = "cuda:0" if torch.cuda.is_available() else "cpu"

# Load model with specific device mapping to ensure single GPU usage
model = AutoModelForCausalLM.from_pretrained(
    finetuned_model_dir,
    device_map={"": target_device},  # Force all modules to same device
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(finetuned_model_dir, trust_remote_code=True)

print(f"Model loaded and placed on {target_device}")
#Load the GSM8K test split
dataset = load_dataset("openai/gsm8k", "main", split="test")
num_examples=len(dataset)


# Force model to use a single device
model = model.to(target_device)

results = []
total = 0
correct = 0
system_prompt = "Please reason step by step, and conclude your response with 'Answer: [final answer]'."

    
for example in tqdm(dataset.select(range(num_examples)), desc="Evaluating GSM8K"):
    question = example["question"]
    ground_truth_answer = example["answer"].split("####")[-1].strip()
    try:
        ground_truth_answer=float(ground_truth_answer)
    except:
        ground_truth_answer=ground_truth_answer
    
    # Construct chat-style prompt using instruct format
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question}
    ]
    prompt_text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    
    try:
        # Clear CUDA cache to free up memory
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        
        # Make sure inputs are on the target device
        inputs = tokenizer(prompt_text, return_tensors="pt").to(target_device)
        
        with torch.no_grad():
            # Generate the outputs
            outputs = model.generate(
                **inputs,
                max_new_tokens=512,
                temperature=0.0,
                do_sample=False
            )
        
        # Make sure outputs are on the target device if needed
        if outputs.device != target_device:
            outputs = outputs.to(target_device)
            
        generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
        predicted_answer = extract_final_answer(generated)
        
        
        # Normalize answers for comparison
        
        num_predicted = extract_boxed_content(predicted_answer)
        

        #num_predicted=float(num_predicted)
        #ground_truth = float(ground_truth_answer)
            

        if isinstance(num_predicted, str) and isinstance(ground_truth_answer, str):
            if compare_ignore_spaces(num_predicted, ground_truth_answer):
                is_correct=True
                correct +=1
            else:
                is_correct=False
        else:
            if num_predicted== ground_truth_answer:
                is_correct=True
                correct +=1
            else: 
                is_correct=False
        
        results.append({
            "question": question,
            "ground_truth": ground_truth_answer,
            "predicted": predicted_answer,
            "normalized_predicted": num_predicted,
            "is_correct": is_correct
        })
        
        print(f"Example {total + 1}:")
        print("Question:", question)
        print("Ground Truth:", ground_truth_answer)
        print("num_predicted:", num_predicted)
        print("Predicted:", predicted_answer)
        print("Correct:", is_correct)
        print("-" * 50)
        
    except Exception as e:
        print(f"Error processing example {total + 1}: {e}")
        results.append({
            "question": question,
            "ground_truth": ground_truth_answer,
            "predicted": "ERROR",
            "is_correct": False,
            "error": str(e)
        })
    
    total += 1

accuracy = correct / total if total > 0 else 0
print(f"GSM8K Accuracy on {total} examples: {accuracy:.4f}")

# Save results to file
with open("gsm8k_results.json", "w") as f:
    json.dump({
        "accuracy": accuracy,
        "examples": results
    }, f, indent=2)






Starting model evaluation on GSM8K and MATH datasets...
PyTorch version: 2.5.1+cu121
CUDA available: True
CUDA device count: 2
Current CUDA device: 0
CUDA device name: Tesla T4


config.json:   0%|          | 0.00/656 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.32k [00:00<?, ?B/s]

...

Example 1317:
Question: Mark's car breaks down and he needs to get a new radiator.  The cost for a new radiator is $400 but he goes to get it at a junk shop and gets it for 80% off.  He then hires a mechanic to install it and it takes 3 hours at $50 an hour.  How much did he pay?
Ground Truth: 230.0
num_predicted: 230.0
Predicted: [final answer]'.
user
Mark's car breaks down and he needs to get a new radiator.  The cost for a new radiator is $400 but he goes to get it at a junk shop and gets it for 80% off.  He then hires a mechanic to install it and it takes 3 hours at $50 an hour.  How much did he pay?
assistant
To determine the total cost Mark paid for the new radiator, we need to break down the expenses step by step.

1. **Calculate the discounted price of the radiator:**
   The original price of the radiator is $400. The junk shop offers an 80% discount.
   \[
   \text{Discount} = 400 \times 0.80 = 320
   \]
   \[
   \text{Discounted price} = 400 - 320 = 80
   \]

2. **Calculate t

Evaluating GSM8K: 100%|█████████▉| 1318/1319 [3:33:32<00:10, 10.48s/it]

Example 1318:
Question: Farmer Brown has 20 animals on his farm, all either chickens or cows. They have a total of 70 legs, all together. How many of the animals are chickens?
Ground Truth: 5.0
num_predicted: 5.0
Predicted: [final answer]'.
user
Farmer Brown has 20 animals on his farm, all either chickens or cows. They have a total of 70 legs, all together. How many of the animals are chickens?
assistant
To determine how many of Farmer Brown's animals are chickens, we can set up a system of equations based on the information given. Let's define the variables first:

- Let \( c \) be the number of chickens.
- Let \( k \) be the number of cows.

We know two things:
1. The total number of animals is 20.
2. The total number of legs is 70.

From the first piece of information, we can write the equation:
\[ c + k = 20 \]

From the second piece of information, we know that each chicken has 2 legs and each cow has 4 legs, so we can write the equation:
\[ 2c + 4k = 70 \]

We now have a system o

Evaluating GSM8K: 100%|██████████| 1319/1319 [3:33:40<00:00,  9.72s/it]

Example 1319:
Question: Henry and 3 of his friends order 7 pizzas for lunch. Each pizza is cut into 8 slices. If Henry and his friends want to share the pizzas equally, how many slices can each of them have?
Ground Truth: 14.0
num_predicted: 14.0
Predicted: [final answer]'.
user
Henry and 3 of his friends order 7 pizzas for lunch. Each pizza is cut into 8 slices. If Henry and his friends want to share the pizzas equally, how many slices can each of them have?
assistant
To determine how many slices of pizza each person can have, we need to follow these steps:

1. Calculate the total number of pizza slices.
2. Determine how many people are sharing the pizza.
3. Divide the total number of slices by the number of people.

First, we calculate the total number of pizza slices. Since each pizza is cut into 8 slices and there are 7 pizzas, we multiply the number of pizzas by the number of slices per pizza:
\[ 7 \text{ pizzas} \times 8 \text{ slices per pizza} = 56 \text{ slices} \]

Next, we d